**库的import和安装**

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torchvision import datasets, transforms, models 
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split

**网络定义**

In [ ]:
net = models.alexnet = models.alexnet(pretrained=True)
net

In [ ]:
net.classifier = nn.Sequential(
    nn.Linear(9216,1024),
    nn.ReLU(),
    nn.Dropout(p=0.7),
    nn.Linear(1024,5),
    nn.LogSoftmax(dim=1)
)

In [ ]:
net

**数据集的准备**

In [ ]:
batch_size = 512

In [ ]:
train_transform=transforms.Compose([
        transforms.RandomRotation(10),      # rotate +/- 10 degrees
        transforms.RandomHorizontalFlip(),  # reverse 50% of images
        transforms.Resize(224),             # resize shortest side to 224 pixels
        transforms.CenterCrop(224),         # crop longest side to 224 pixels at center
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
])

In [ ]:
dataset=datasets.ImageFolder(root=("../input/flowers-recognition/flowers"),transform=train_transform)
dataset

In [ ]:
class_names=dataset.classes
print(class_names)
print(len(class_names))

In [ ]:
train_indices, test_indices = train_test_split(list(range(len(dataset.targets))), test_size=0.2, stratify=dataset.targets)
train_data = torch.utils.data.Subset(dataset, train_indices)
test_data = torch.utils.data.Subset(dataset, test_indices)

In [ ]:
train_loader=DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test_data,batch_size=batch_size)

In [ ]:
print(len(train_data))
print(len(test_data))

**训练**

In [ ]:
def train(net,train_iter, test_iter, num_epochs, lr, device):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.classifier.apply(init_weights)
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.Adam(net.parameters(),lr=lr)
    loss = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        start_time=time.time()
        train_accuracy = 0.
        l_mean = 0.
        test_accuracy = 0.
        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            
            y_hat = net(X)
            l = loss(y_hat, y)
            
            train_acc = 0.
            train_predicted=torch.max(y_hat.data,1)[1]
            train_acc_number = (train_predicted == y).sum().item()
            train_acc = train_acc_number / y.shape[0]
            train_accuracy += train_acc
            
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            
            with torch.no_grad():
                l_mean += l.item()
                #print('i : ',i + 1,'l_mean : ',l_mean,'l.item() : ',l.item())
            with torch.no_grad():
                for ii, (X, y) in enumerate(test_iter):
                    X, y = X.to(device), y.to(device)
                    y_hat = net(X)
                    predicted=torch.max(y_hat.data,1)[1]
                    
                    test_acc = 0
                    test_acc_num = (predicted == y).sum().item()
                    test_acc = test_acc_num / y.shape[0] 
                    test_accuracy += test_acc
                    
        test_accuracy = test_accuracy / (ii + 1) / (i + 1)
        l_mean = l_mean / (i + 1)
        train_accuracy = train_accuracy / (i + 1)


        print("epoch : ",epoch + 1,'loss : ',l_mean,"train_acc : ",train_accuracy,'test_acc : ',test_accuracy,
             "time : ",time.time() - start_time)

In [ ]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')
device = try_gpu()

In [ ]:
epoch = 1000
lr = 0.0001
train(net,train_loader,test_loader,epoch,lr,device)

**推理**

In [ ]:
for X,y in test_loader:
    break

In [ ]:
inv_normalize=transforms.Normalize(mean=[-0.485/0.229,-0.456/0.224,-0.406/0.225],
                                     std=[1/0.229,1/0.224,1/0.225])
im = inv_normalize(X[0])
plt.imshow(np.transpose(im.numpy(),(1,2,0)))

In [ ]:
X[0].shape

In [ ]:
net.eval()
pred = []
with torch.no_grad():
    for i in range(10):
        new_pred=net(X[i].reshape(1,3,224,224).to(device)).argmax()
        pred.append(new_pred)

In [ ]:
pred,y[0:10]

In [ ]:
class_names[pred[2].item()],class_names[y[2].item()]

In [ ]:
im = inv_normalize(X[2])
plt.imshow(np.transpose(im.numpy(),(1,2,0)))

In [ ]:
class_names[pred[8].item()],class_names[y[8].item()]

In [ ]:
im = inv_normalize(X[8])
plt.imshow(np.transpose(im.numpy(),(1,2,0)))

In [ ]:
torch.save(net, 'Flowers Recognition_fine_tuning_test_acc_0.87.params')